In [ ]:
import numpy as np
import pandas as pd
from neurokit2.ppg import ppg_simulate
from gcp_utils.tools.utils import resample_frame, split_frame

pd.options.plotting.backend = 'plotly'

In [ ]:
x = ppg_simulate(duration=20.48, sampling_rate=200).astype(np.float32).tolist()

In [ ]:
from gcp_utils import constants

processed_frame, pulse_rate, spo2, r = constants.processed_frame()
pulse_rate, spo2, r

In [ ]:
import numpy as np
import pickle as pkl
from typing import Tuple
from database_tools.preprocessing.datastores import ConfigMapper, Window
from database_tools.preprocessing.functions import bandpass
from database_tools.preprocessing.utils import resample_signal

def process_frame(red_frame, ir_frame, config):
    cm = ConfigMapper(config)
    red_filt = bandpass(red_frame, low=cm.freq_band[0], high=cm.freq_band[1], fs=cm.bpm_fs)
    ir_filt = bandpass(ir_frame, low=cm.freq_band[0], high=cm.freq_band[1], fs=cm.bpm_fs)

    red_filt_flip = _flip_signal(red_filt)
    ir_filt_flip = _flip_signal(ir_filt)
    combined = (red_filt_flip + ir_filt_flip) / 2  # averaging strategy

    combined_resamp = resample_signal(sig=combined.tolist(), fs_old=cm.bpm_fs, fs_new=cm.fs)

    windows = _split_frame(sig=combined_resamp, n=int(combined_resamp.shape[0] / cm.win_len))
    result = {
        'red_frame_for_processing': list(red_filt),
        'ir_frame_for_processing': list(ir_filt),
        'red_frame_for_presentation': list(red_filt_flip),
        'ir_frame_for_presentation': list(ir_filt_flip),
        'combined_frame_for_presentation': list(combined),
        'windows': windows,
    }
    return result

def _flip_signal(sig):
    """Flip signal data but subtracting the maximum value."""
    flipped = np.max(sig) - sig
    return flipped

def _split_frame(sig: np.ndarray, n: int) -> list:
    """Split list into n lists.

    Args:
        sig (list): Data.
        n (int): Number of lists.

    Returns:
        n_sigs (list): Data split in to n lists.
    """
    n_sigs = [s.tolist() for s in np.split(sig, n)]
    return n_sigs

In [ ]:
from gcp_utils import constants

red_frame = constants.BPM_FRAME['red_frame']
ir_frame = constants.BPM_FRAME['ir_frame']
process_frame(red_frame, ir_frame, constants.CONFIG)